In [11]:
#import python libraries  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import sys
import os
import time
import scipy
import pickle
from importlib import reload

codebase_path = '/data/home/wpw035/Codebase'
sys.path.insert(0, codebase_path) #add path to my codebase models

In [15]:
#my libary imports, code in DRP_nb file
from DRP_nb import data_imports, splitting, hp_opt, utils
reload(data_imports)
reload(splitting)
reload(utils)

<module 'DRP_nb.utils' from '/data/home/wpw035/Drug_response_prediction/DRP_exmaple_msc/drug_response_prediction_example/DRP_nb/utils.py'>

## Data Imports
The data loading and cleaning are done within the data_imports module within my DRP_nb library. The data loading and cleaning are done within the data_imports module within my DRP_nb library. Take a look at the data_imports file within DRP_nb to see all the steps. 

In [13]:
#omics and target imports 
prot, rna, phospho_ls, ic50_df1, one_hot_cls, one_hot_drugs = data_imports.read_inputs_phos_ect()
_all_cls = prot.index
_all_drugs = one_hot_drugs.columns


#load landmark genes that will be used for feature selection
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


% of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


## Feature selection 
Feature selection using the 1K landmark genes

will foucs on RNA-seq omics data here for simplicity. 
Ony keep an feature from the rna df if it is an landmark gene

In [30]:
#finds landmarks that are also in rna df
overlapping_landmarks = [g for g in landmark_genes.index if g in rna.columns]
rna = rna[overlapping_landmarks]

In [33]:
print(rna.shape)
rna.head()

(38, 908)


GENE_SYMBOLS,INSIG1,FOXO3,CDH3,ORC1,ITGAE,CDK7,FOS,CDK6,CDK4,CDK2,...,FHL2,IL13RA1,FKBP4,ABHD4,HIST1H2BK,NUCB2,NUP88,NVL,ILK,INPP1
COLO-680N,8.154784,9.236981,9.799767,5.103765,10.757260,9.690477,9.034680,6.728485,8.812277,6.160667,...,8.719871,6.707274,7.683773,3.787795,9.465148,7.516577,7.447703,6.983274,6.795911,9.728236
GDM-1,8.055311,8.088824,3.379974,6.465721,9.480014,8.702067,3.157377,9.285379,8.979623,6.235116,...,6.329245,4.010940,9.105476,3.394924,8.919325,8.434682,7.330557,6.003414,6.626827,8.023713
HEL,7.678645,8.126148,3.383949,7.070015,8.819721,8.586132,6.420190,8.927892,9.720554,6.398864,...,9.807979,3.422834,7.842642,3.879684,11.686403,7.181875,6.733916,6.101298,6.616129,9.701949
HL-60,9.919567,7.989809,3.171409,7.077472,8.133847,8.347510,6.516647,8.619618,9.546260,6.068342,...,3.166125,4.936508,7.779411,3.562882,11.235920,10.315398,6.815571,6.391719,6.212755,5.730575
JHH-2,9.251225,8.227397,3.260621,3.861217,9.561528,8.988581,5.048408,8.471914,8.982768,5.421258,...,9.257869,8.454283,6.429325,5.200694,9.746995,8.678383,6.247309,6.080807,7.977128,8.423243


The above shows the rna df has 38 rows and 908 columns. Thus, it gives the expression values for 38 cell lines for 908 landmark genes that were in the original rna-seq dataset. 

The input we want for drug response prediction is drug cell line pairs because we want to predict the efficacy of a given drug for a given cell.


The input we want for drug response prediction is drug cell line pairs because we want to predict the efficacy of a given drug for a given cell. Therefore, the next step is to use the df we have to create inputs for all drug cell line pairs. 

## Drug cell line input data creation.

Creates inputs for each drug cell line pair using the create_all_drugs fuction,
have a look in the utils module to see the create_all_drugs function.

In [41]:
#create input data for each drug
x_all, x_drug, y_series = utils.create_all_drugs(
    rna, one_hot_drugs, ic50_df1, _all_cls)

x_all = x_all.astype(np.float32)
x_drug = x_drug.astype(np.float16)

#fmt index to include drug cell line paris
cls_drugs_index = x_all.index + '::' + x_drug.index
x_all.index = cls_drugs_index
x_drug.index = cls_drugs_index
y_series.index = cls_drugs_index

x_all.shape, x_drug.shape, len(y_series)

((11583, 908), (11583, 345), 11583)

We now have three objects for use in our models. Two input dataframes, x_all and x_drug. x_all gives the rna-seq data for each drug cell line pair, and x_drug gives the one-hot representation of the drugs. y_series is a pandas series object that gives the target IC50 values for each drug cell line pair. 

Notice that the index of the dataframes match up with the target values. Such that x_all.iloc[i], x_drug.iloc[i] and y_series[i] refer to the same drug cell line pair. 

In [45]:
x_all.head()

GENE_SYMBOLS,INSIG1,FOXO3,CDH3,ORC1,ITGAE,CDK7,FOS,CDK6,CDK4,CDK2,...,FHL2,IL13RA1,FKBP4,ABHD4,HIST1H2BK,NUCB2,NUP88,NVL,ILK,INPP1
COLO-680N::(5Z)-7-Oxozeaenol,8.154784,9.236981,9.799767,5.103765,10.757260,9.690476,9.034679,6.728485,8.812278,6.160667,...,8.719871,6.707273,7.683773,3.787795,9.465148,7.516577,7.447703,6.983274,6.795911,9.728236
GDM-1::(5Z)-7-Oxozeaenol,8.055311,8.088824,3.379974,6.465721,9.480014,8.702067,3.157377,9.285379,8.979623,6.235116,...,6.329245,4.010940,9.105475,3.394924,8.919325,8.434682,7.330557,6.003414,6.626827,8.023713
HEL::(5Z)-7-Oxozeaenol,7.678645,8.126147,3.383949,7.070015,8.819721,8.586131,6.420190,8.927892,9.720554,6.398864,...,9.807980,3.422834,7.842642,3.879684,11.686403,7.181875,6.733916,6.101298,6.616129,9.701949
HL-60::(5Z)-7-Oxozeaenol,9.919567,7.989809,3.171409,7.077472,8.133847,8.347510,6.516647,8.619617,9.546260,6.068342,...,3.166125,4.936508,7.779411,3.562882,11.235920,10.315398,6.815571,6.391719,6.212755,5.730575
JHH-2::(5Z)-7-Oxozeaenol,9.251225,8.227397,3.260621,3.861217,9.561527,8.988581,5.048408,8.471914,8.982768,5.421257,...,9.257870,8.454283,6.429326,5.200694,9.746995,8.678383,6.247309,6.080807,7.977128,8.423243


In [46]:
x_drug.head()

,0,1,2,3,4,5,6,7,8,9,...,335,336,337,338,339,340,341,342,343,344
COLO-680N::(5Z)-7-Oxozeaenol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GDM-1::(5Z)-7-Oxozeaenol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HEL::(5Z)-7-Oxozeaenol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HL-60::(5Z)-7-Oxozeaenol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JHH-2::(5Z)-7-Oxozeaenol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
y_series.head()

COLO-680N::(5Z)-7-Oxozeaenol    1.729743
GDM-1::(5Z)-7-Oxozeaenol       -0.523250
HEL::(5Z)-7-Oxozeaenol          0.948509
HL-60::(5Z)-7-Oxozeaenol        1.426720
JHH-2::(5Z)-7-Oxozeaenol        1.250863
dtype: float64